In [3]:
import pandas as pd 
from bondnet.dataset.mg_barrier import process_species
path_mg_data = "/home/santiagovargas/Documents/Dataset/mg_dataset/"
path_json = path_mg_data + "20220613_reaction_data.json"
mg_df = pd.read_json(path_json)
print(mg_df[mg_df['product_id'] == '62707'])
#for index, row in mg_df.iterrows():
#    try:
#        if(len(row['product_energy']) > 1): 
#            print(row['product_energy'])
#    except:
#        pass
#print(mg_df.iloc[1]['product_energy'])
#mg_df.iloc[0]["product_structure"]
#row["reactant_bonds"]
#for ind, row in mg_df.iterrows(): 
#    if(row["product_molecule_graph"]['graphs']['multigraph'] == False):
#        print(row["product_molecule_graph"]['graphs']['multigraph'])
#mg_df.iloc[0]["product_molecule_graph"]['graphs']
#print(mg_df.iloc[0]['reactant_structure'])

mg_df.iloc[0]['reactant_molecule_graph']

/home/santiagovargas/anaconda3/envs/bondnet/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using backend: pytorch


Empty DataFrame
Columns: [_id, builder_meta, charge, spin_multiplicity, natoms, elements, nelements, composition, formula_alphabetical, chemsys, symmetry, reaction_id, reactant_id, product_id, transition_state_id, deprecated, reactant_structure, reactant_molecule_graph, reactant_molecule_graph_nometal, reactant_bonds, reactant_bonds_nometal, reactant_energy, reactant_zpe, reactant_enthalpy, reactant_entropy, reactant_free_energy, product_structure, product_molecule_graph, product_molecule_graph_nometal, product_bonds, product_bonds_nometal, product_energy, product_zpe, product_enthalpy, product_entropy, product_free_energy, transition_state_structure, transition_state_energy, transition_state_zpe, transition_state_enthalpy, transition_state_entropy, transition_state_free_energy, dE, dH, dS, dG, dE_barrier, dH_barrier, dS_barrier, dG_barrier, bonds_broken, bond_types_broken, bonds_broken_nometal, bond_types_broken_nometal, bonds_formed, bond_types_formed, bonds_formed_nometal, bond_type

{'@module': 'pymatgen.analysis.graphs',
 '@class': 'MoleculeGraph',
 'molecule': {'@module': 'pymatgen.core.structure',
  '@class': 'Molecule',
  'charge': 0,
  'spin_multiplicity': 1,
  'sites': [{'name': 'O',
    'species': [{'element': 'O', 'occu': 1}],
    'xyz': [-3.6939830000000002, -1.816491, 1.754168],
    'properties': {}},
   {'name': 'C',
    'species': [{'element': 'C', 'occu': 1}],
    'xyz': [-3.425275, -2.754684, 1.1204],
    'properties': {}},
   {'name': 'C',
    'species': [{'element': 'C', 'occu': 1}],
    'xyz': [2.747296, 2.115733, -2.342419],
    'properties': {}},
   {'name': 'C',
    'species': [{'element': 'C', 'occu': 1}],
    'xyz': [3.29943, 2.470152, -1.176623],
    'properties': {}},
   {'name': 'C',
    'species': [{'element': 'C', 'occu': 1}],
    'xyz': [-3.1194, -3.8097000000000003, 0.399676],
    'properties': {}},
   {'name': 'C',
    'species': [{'element': 'C', 'occu': 1}],
    'xyz': [2.667411, 2.234613, 0.09411499999999999],
    'properties': {}}

In [87]:
#mg_df[mg_df['reactant_id'] == 62707]
#print(len(mg_df.iloc[0]['product_molecule_graph']['graphs']['nodes']))
#print(mg_df.iloc[0]['product_molecule_graph']['graphs']['adjacency'])
bonds_net = []
for ind, i in enumerate(mg_df.iloc[0]['product_molecule_graph']['graphs']['adjacency']):
    for j in i: 
        bonds_net.append([ind, j['id']]) 
        
print(bonds_net)
print(mg_df.iloc[0]['product_bonds'])

[[0, 1], [1, 2], [1, 4], [2, 7], [2, 3], [2, 8], [3, 9], [3, 4], [3, 5], [4, 11], [4, 10], [5, 6]]
[[0, 1], [1, 2], [1, 4], [2, 7], [2, 3], [2, 8], [3, 9], [3, 4], [3, 5], [4, 11], [4, 10], [5, 6]]


In [3]:
bond_count = 0
for ind, row in mg_df.iterrows():
    if(len(row['bonds_broken']) == 1):bond_count+=1
    elif(len(row['bonds_broken_nometal']) == 1):bond_count+=1
    elif(len(row['bonds_formed_nometal'])!=1 ):bond_count+=1
    elif(len(row['bonds_formed'])!= 1):bond_count+=1
    else:bond_count+=0
print(bond_count / ind)

0.8103333333333333


In [8]:
import pandas as pd

path_mg_data = "/home/santiagovargas/Documents/Dataset/mg_dataset/"
path_json = path_mg_data + "20220613_reaction_data.json"
mg_df = pd.read_json(path_json)



from bondnet.utils import int_atom, xyz2mol
from bondnet.core.molwrapper import rdkit_mol_to_wrapper_mol
atoms = [int_atom(atom['name']) for atom in mg_df.iloc[0]['reactant_molecule_graph']["molecule"]["sites"]]
xyz_coordinates = [i["xyz"] for i in mg_df.iloc[0]['reactant_molecule_graph']["molecule"]["sites"]]
charge = int(mg_df.iloc[0]['charge'])
mol = xyz2mol(atoms = atoms, 
                coordinates = xyz_coordinates, 
                charge = charge,
                )
print(rdkit_mol_to_wrapper_mol(mol[0], identifier= str(mg_df.iloc[0]["reactant_id"])))

-8228.114932717173
-0.0005337600468919845
-0.0005337600468910001
0.0922205206061335
0.092220520606133
-8226.321824156197
62707_C5H5N1O1


In [51]:
from pymatgen.core import Lattice, Structure, Molecule

#[i for i in mg_df.iloc[1]['product_structure']]
#print(mg_df.iloc[1][""])



_id                                {'@module': 'bson.objectid', '@class': 'Object...
builder_meta                       {'emmet_version': '0.1.1a1.dev210+gbd28b91d', ...
charge                                                                             0
spin_multiplicity                                                                  1
natoms                                                                           NaN
elements                                                                [C, H, N, O]
nelements                                                                          4
composition                                 {'C': 3.0, 'N': 2.0, 'O': 1.0, 'H': 6.0}
formula_alphabetical                                                     C3 H6 N2 O1
chemsys                                                                      C-H-N-O
symmetry                           {'point_group': 'Cs', 'rotation_number': 1.0, ...
reaction_id                                                      

In [37]:
from bondnet.core.molwrapper import create_wrapper_mol_from_atoms_and_bonds
from bondnet.core.reaction import Reaction 
from bondnet.core.reaction_collection import ReactionCollection

row = mg_df.iloc[1]
barrier = row["dE_barrier"]
species = [i['name'] for i in row['product_molecule_graph']["molecule"]["sites"]]
coords = [i["xyz"] for i in row['product_molecule_graph']["molecule"]["sites"]]
bonds = row['product_bonds']

charge = row['charge']
free_energy = row['product_free_energy']
id = str(row["reactant_id"])

product = create_wrapper_mol_from_atoms_and_bonds(
    species, coords, bonds, charge=0, free_energy=free_energy, identifier=id,
)


species = [i['name'] for i in row['reactant_molecule_graph']["molecule"]["sites"]]
coords = [i["xyz"] for i in row['reactant_molecule_graph']["molecule"]["sites"]]
bonds = row['reactant_bonds']
charge = row['charge']

free_energy = row['reactant_free_energy']
id = str(row["product_id"])
reactant = create_wrapper_mol_from_atoms_and_bonds(
    species, coords, bonds, charge=0, free_energy=free_energy, identifier=id,
)

id = [i for i in row["reaction_id"].split("-")]
id = int(id[0] + id[1] + id[2])
broken_bond = row['bonds_formed']
if(broken_bond == []):broken_bond=None
rxn = Reaction(
            reactants = [reactant], 
            products=[product], 
            free_energy= barrier,
            broken_bond = broken_bond,
            identifier = id
            )

extractor = ReactionCollection([rxn])
reactant_bond_list = list(reactant.bonds.keys())
product_bond_list = list(product.bonds.keys())
reactant_bond_list.sort()
product_bond_list.sort()
#print(reactant_bond_list)
#print(product_bond_list)
#print(row["bonds_broken_nometal"])
#print(row["bonds_broken"])
#print(row["bonds_formed"])
#print(row["bonds_formed_nometal"])


print(rxn)
extractor.create_regression_dataset_reaction_network_simple(
    struct_file=path_mg_data + "mg_struct_bond_rgrn.sdf",
    label_file=path_mg_data + "mg_label_bond_rgrn.yaml",
    feature_file=path_mg_data + "mg_feature_bond_rgrn.yaml"
)



A -> B style reaction
reactants:
    C3H6N2O1 (0)
products:
    C3H6N2O1 (0)



RuntimeError: Cannot break a reactant bond to get products; invalid reaction: Reaction id: 652504985965251. Reactants id: 65251 Products id: 65250 

In [33]:
from pymatgen.analysis.graphs import MoleculeGraph
from bondnet.core.reaction import Reaction

reactant = MoleculeGraph(mg_df.iloc[1]['reactant_molecule_graph']['molecule'])
#product = MoleculeGraph()

print(mg_df.iloc[1]['reactant_molecule_graph']['molecule'])
print(len(mg_df.iloc[1]['reactant_molecule_graph']['molecule']))

#row['reactant_bonds']
#rxn = Reaction(reactants = ,
#                products = ,
#                broken_bond = mg_df.iloc[1]['bonds_broken'],
#                free_energy = mg_df.iloc[1]['dE_barrier'],
#                identifier = mg_df.iloc[1]['reaction_id'])

AttributeError: 'NoneType' object has no attribute 'get'

In [1]:
'''
molecule_attributes
simple the charge, in order 
molecules.sdf <--simply barf all elements from products/reactants to thisis 
this is vanilla sdf 

reactions.yaml
- energy: 2.5077022141203997 <--- transition_state_energy
  index: 0 <-- keep evan's reaction id here 
  products: <--- will need to remap these
  - 186 
  - 38
  reactants: <-- will need to remap these
  - 45
'''


  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)


In [7]:
from bondnet.dataset.mg_barrier import *
create_struct_label_dataset_bond_based_regression(filename='', out_file='./')


0it [00:00, ?it/s]RDKit ERROR: [08:56:55] SMARTS Parse Error: syntax error while parsing: [#O]
RDKit ERROR: [08:56:55] SMARTS Parse Error: Failed parsing SMARTS '[#O]' for input: '[#O]'
0it [00:00, ?it/s]

0


[08:56:55] SMARTS Parse Error: syntax error while parsing: [#O]

[08:56:55] SMARTS Parse Error: Failed parsing SMARTS '[#O]' for input: '[#O]'


ArgumentError: Python argument types in
    RWMol.__init__(RWMol, NoneType)
did not match C++ signature:
    __init__(_object*, RDKit::ROMol, bool, int)
    __init__(_object*, RDKit::ROMol, bool)
    __init__(_object*, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >)
    __init__(_object*)
    __init__(_object*, RDKit::ROMol)

In [ ]:
def atom_mapping(self):
    """
    Find the atom mapping between products and reactant.

    For example, suppose we have reactant

            C 0
            / \
        /___\
        O     N---H
        1     2   3

    products
            C 0
            / \
        /___\
        O     N
        1     2
    and (note the index of H changes it 0)
        H 0

    This function generates the atom mapping:
    [{0:0, 1:1, 2:2}, {0:3}]
    where the first dict is the mapping for product 1 and the second dict is the
    mapping for product 2.

    Returns:
        list: each element is a dict that maps the atoms from product to reactant.
    """
    if self._atom_mapping is not None:
        return self._atom_mapping

    # get subgraphs of reactant by breaking the bond
    # if A-> B reaction, there is one element in subgraphs
    # if A-> B+C reaction, there are two
    
    bond = self.get_broken_bond()
    original = copy.deepcopy(self.reactants[0].mol_graph)
    original.break_edge(bond[0], bond[1], allow_reverse=True)
    components = nx.weakly_connected_components(original.graph)
    subgraphs = [original.graph.subgraph(c) for c in components]

    # correspondence between products and reactant subgrpahs
    if len(subgraphs) == 1:
        corr = {0: 0}
    else:
        # product idx as key and reactant subgraph idx as value
        # order matters since mappings[0] (see below) corresponds to first product
        corr = OrderedDict()
        products = [p.mol_graph for p in self.products]

        # implicitly indicating _isomorphic(subgraphs[1], products[1].graph)
        if _isomorphic(subgraphs[0], products[0].graph):
            corr[0] = 0
            corr[1] = 1
        else:
            corr[0] = 1
            corr[1] = 0

    # atom mapping between products and reactant
    mappings = []
    for pidx, fidx in corr.items():
        mp = nx_graph_atom_mapping(self.products[pidx].graph, subgraphs[fidx])
        if mp is None:
            raise RuntimeError(f"cannot find atom mapping for reaction {str(self)}")
        else:
            mappings.append(mp)

    self._atom_mapping = mappings

    return self._atom_mapping

In [16]:
#reactant_structure, reactant_molecule_graph, reactant_bonds, 
#product_structure, product_molecule_graph, product_bonds, 
#transition_state_structure, 
#bonds_broken, bonds_broken_nometal, bonds_formed, bonds_formed_nometal
from tqdm import tqdm 
from bondnet.utils import to_path, int_atom, xyz2mol
from bondnet.core.molwrapper import rdkit_mol_to_wrapper_mol, create_wrapper_mol_from_atoms_and_bonds


element_dict = {"H": 1, "Li": 3, "Be": 4, "B": 5, "C": 6, "N": 7, "O": 8, 
"F": 9, 'Ca': 20, 'Na': 11, 'Cl': 17, 'P':15, 'Mg': 12 }
preserve = 0
for i in tqdm(range(2000)):
    row = mg_df.iloc[i]
    """g_prod = nx.Graph(row["product_bonds"])
    g_react = nx.Graph(row["reactant_bonds"])
    g_prod_2 = nx.Graph(row["product_bonds_nometal"])
    g_react_2 = nx.Graph(row["reactant_bonds_nometal"])
    if(nx_graph_atom_mapping(g_prod, g_react) == None):
        if(nx_graph_atom_mapping(g_prod_2, g_react_2) == None):
            pass
        else: works_count += 1 
    else: works_count += 1"""
    species_reactant = [int_atom(i['name']) for i in row['reactant_molecule_graph']["molecule"]["sites"]]
    species_products = [int_atom(i['name']) for i in row['product_molecule_graph']["molecule"]["sites"]]
    trans_list_temp = list(i['name'] for i in list(row["transition_state_structure"]['sites']))
    trans_list = [element_dict[i] for i in trans_list_temp]
    
    if(species_products == trans_list): preserve += 1
    #if(list_xyz == coords_reactant): preserve += 1

print(preserve)



100%|██████████| 2000/2000 [00:00<00:00, 9615.40it/s]

2000
